In [7]:
import pandas as pd
import numpy as np

In [2]:
path = "C:/Users/Artem/Documents/Diploma/output/links_collection/"
read_list =[path + 'data04_02_2017__02_14_35_STEP_ONE.csv',
           path + 'FINAL_data04_02_2017__05_12_02.csv',
           path + 'FINAL_data04_02_2017__10_21_52.csv']

In [5]:
df = pd.DataFrame()
for file in read_list:
    df_mini = pd.read_csv(file,sep=';')
    if len(df)!=0:
        df = pd.concat([df,df_mini])
    else:
        df = df_mini.copy()

In [8]:
df['min_price'] = np.nan
df['max_price'] = np.nan
df['min_price_date'] = np.nan
df['max_price_date'] = np.nan

In [10]:
df.to_csv('all_links_4_02_2016.csv',sep=';')

In [1]:
#get_data_from_page_PART3.py NEW!!!
import pandas as pd
import requests
import re
def html_stripper(text):
    return re.sub('<[^<]+?>', '', str(text))
from bs4 import BeautifulSoup


def if_closed(flat_page, dic):
    sth = flat_page.find('span', attrs={'class':"object_descr_warning object_descr_warning_red"})
    sth = html_stripper(sth)
    sth = sth.replace('\n',' ')
    sth = sth.replace('                       ',' ')
    dic['if_closed'] = sth
def get_price_ar(flat_page, dic):
    sth = flat_page.find('div', attrs={'id':"price_rur"})
    sth = html_stripper(sth)
    sth = sth.replace('\n',' ')
    sth = sth.replace('                       ',' ')
    dic['price_ar'] = sth

#Выдаёт не один элемент, а сразу небольшой словарь с переменными
def metro(flat_page,dic):
    #в sth (от англиского something) содержится много того, что нам нужно   
    # сколько миниут до метро и на машине или пешком
    sth = flat_page.find('span', attrs={'class':'object_item_metro_comment'})
    sth = html_stripper(sth)
    sth_list=sth.split('\n')
    sth_list = list(filter(None, sth_list))
    Metrdist = re.findall('\d+', sth)
    if len(Metrdist)==1:
        dic['Metro_time'] = int( Metrdist[0] )
    if 'пешком'in sth:
        dic['Walk_metro']=int(1)
    if 'машин' in sth:
        dic['Walk_metro']=int(0)
    # Станция метро
    sth = flat_page.find('a', attrs={'class':"object_item_metro_name"})
    sth = html_stripper(sth)
    dic["Metro_station"] = str(sth)
    
def add_feat4(dic,sth,spliter1,spliter2, key):
    #sth_list =str(sth).split(spliter)
    sth_list = re.split(spliter1+'|'+spliter2,sth)
    if len(sth_list)>1:
        dic[key] = sth_list[1]
    else:
        dic[key] = sth_list[0]
def get_address(flat_page,dic):
    sth = flat_page.find('h1', attrs={'class':'object_descr_addr'})
    sth = html_stripper(sth)
    sth = sth.replace('\n',' ')
    sth = sth.replace('                       ',' ')
    dic['Address'] = sth
def many_get3(flat_page):
    #print(flat_page)
    dic ={}
    #Всякая хрень почти из подзаголовка
    sth = flat_page.find('span', attrs={'class':'object_descr_dt_added'})#.contents[1]
    sth=str(sth)
    #print(sth)
    add_feat4(dic=dic ,sth=sth ,spliter1='"deal_type": ',spliter2 = '"publication_date"',key= 'sale')
    add_feat4(dic=dic ,sth=sth ,spliter1='"publication_date": ',spliter2 = ', "id":',key= 'publication_date')
    add_feat4(dic=dic ,sth=sth ,spliter1='"is_premium":',spliter2 = '"deal_type":',key= 'is_premium')
    # Добавать ближайшую (первую в списке из ближайших) станций метро, время на способ добраться (пешком или на машине)
    metro(flat_page,dic)
    get_address(flat_page,dic)
    if_closed(flat_page, dic)
    get_price_ar(flat_page, dic)
    
    sth = flat_page.find('div', attrs={'class':'object_descr_text'})
    sth = html_stripper(sth)
    sth.replace('\n',' ')
    #print()
    dic['Description'] = re.sub(' +',' ',sth)
    return(dic)

#link_number = 151535540
#link_number = 150718150
link_number = 147497564
flat_url = 'http://www.cian.ru/sale/flat/' + str(link_number) + '/'
flat_page = requests.get(flat_url)
flat_page = flat_page.content
flat_page = BeautifulSoup(flat_page, 'lxml')
dic={}

print(many_get3(flat_page))
print(list(dic))

{'Description': '\n \n Квартиры в ЖК Прима Парк. Ключи в декабре! От Застройщика. Своя квартира в Новой Москве от 2,8 млн. руб.! Отделка в подарок! Ипотека от 6,5%. Рассрочка 0%. Развитая инфраструктура: сады. школы. ФЗ-214.7 мин. До м. Бунинская аллея.]]&gt;\n \n\n \n\n \n \n \n\n\n\n\n\n\n\n\nКОРТРОС\n\n\n\n\n\n\n\n Показать телефон\n \n\n+7 495 134-60-16\n\n\n Пожалуйста, скажите, что нашли это объявление на ЦИАН\n \n\n\n\n\n\n\n\n Заказ обратного звонка\n \n\n\n\n /* */\n \n', 'price_ar': '3283830,0', 'is_premium': '<span class="object_descr_dt_added">сегодня, 14:38</span>', 'publication_date': '<span class="object_descr_dt_added">сегодня, 14:38</span>', 'Metro_station': 'None', 'sale': '<span class="object_descr_dt_added">сегодня, 14:38</span>', 'Address': ' Москва, Щербинка, район Щербинка, ул. Барышевская Роща, 22 ', 'if_closed': 'None'}
[]
